In [6]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "irisa-ot"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.9.2"

In [2]:
ICEBERG_VERSION

'1.9.2'

In [3]:
SPARK_VERSION


'3.5.6'

In [4]:
SPARK_MINOR_VERSION

'3.5'

# Connect with Spark

In [7]:
config = {
    f"spark.sql.catalog.lakekeeper": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.lakekeeper.type": "rest",
    f"spark.sql.catalog.lakekeeper.uri": CATALOG_URL,
    f"spark.sql.catalog.lakekeeper.warehouse": WAREHOUSE,
    f"spark.sql.catalog.lakekeeper.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "lakekeeper",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}


In [8]:
spark_config = SparkConf().setMaster('spark://spark-master:7077').setAppName("Iceberg-REST-Cluster")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

spark.sql("USE lakekeeper")

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a721a9f4-25b7-4768-9e0f-cb2c318765b9;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.9.2 in central
:: resolution report :: resolve 223ms :: artifacts dl 12ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.9.2 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	--------------------------------------------------------------

DataFrame[]

## Read and Write Tables

In [9]:
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS spark_namespace")
spark.sql("SHOW NAMESPACES").toPandas()

namespace
0  pyiceberg_namespace
1      trino_namespace
2      spark_namespace

In [10]:
data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [11]:
sdf.writeTo(f"spark_namespace.cluster_table").createOrReplace()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".                
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [11]:
spark.sql(f"SELECT * FROM spark_namespace.cluster_table").toPandas()

id   strings  floats
0   1  a-string     2.2

In [12]:
spark.stop()